In [2]:
import bm25s
import Stemmer  # optional: for stemming

# Create your corpus here
corpus = [
    "a cat is a feline and likes to purr",
    "a dog is the human's best friend and loves to play",
    "a bird is a beautiful animal that can fly",
    "a fish is a creature that lives in water and swims",
]



An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [3]:
stemmer = Stemmer.Stemmer("english")

corpus_tokens = bm25s.tokenize(corpus, stopwords = 'en', stemmer = stemmer)

retriever = bm25s.BM25()
retriever.index(corpus_tokens)




Split strings:   0%|          | 0/4 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/4 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/4 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# Query the corpus
query = "does the fish purr like a cat?"
query_tokens = bm25s.tokenize(query, stemmer=stemmer)


Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Get top-k results as a tuple of (doc ids, scores). Both are arrays of shape (n_queries, k)
results, scores = retriever.retrieve(query_tokens, corpus=corpus, k=2)

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
for i in range(results.shape[1]):
    doc, score = results[0, i], scores[0, i]
    print(f"Rank {i+1} (score: {score:.2f}): {doc}")


Rank 1 (score: 1.59): a cat is a feline and likes to purr
Rank 2 (score: 0.48): a fish is a creature that lives in water and swims


In [8]:
# You can save the arrays to a directory...
retriever.save("animal_index_bm25")

In [9]:
# You can save the corpus along with the model
retriever.save("animal_index_bm25", corpus=corpus)

Indexing:   0%|          | 0.00/252 [00:00<?, ?B/s]

In [1]:
import bm25s
reloaded_retriever = bm25s.BM25.load("animal_index_bm25", load_corpus=False)
# set load_corpus=False if you don't need the corpus

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [3]:
import Stemmer  # optional: for stemming
stemmer = Stemmer.Stemmer("english")


In [30]:
# Query the corpus
query = "The human is homosexual"
query_tokens = bm25s.tokenize(query, stemmer=stemmer)
# Get top-k results as a tuple of (doc ids, scores). Both are arrays of shape (n_queries, k)
results, scores = reloaded_retriever.retrieve(query_tokens, k=2)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
results

array([[1, 0]], dtype=int32)

In [32]:
indexes = results[0].tolist()

In [33]:
indexes

[1, 0]

In [34]:
# Create your corpus here
corpus = [
    "a cat is a feline and likes to purr",
    "a dog is the human's best friend and loves to play",
    "a bird is a beautiful animal that can fly",
    "a fish is a creature that lives in water and swims",
]


In [35]:
results = [corpus[i] for i in indexes]
print(results)

["a dog is the human's best friend and loves to play", 'a cat is a feline and likes to purr']


In [57]:
import bm25s
retriever = bm25s.BM25.load("animal_index_bm25", load_corpus=True)
# You can provide a list of queries instead of a single query
queries = ["What is a cat?", "is the bird a dog?"]

# Provide your own stopwords list if you don't like the default one
stopwords = ["a", "the"]

# For stemming, use any function that is callable on each word list
stemmer_fn = lambda lst: [word for word in lst]

# Tokenize the queries
query_token_ids = bm25s.tokenize(queries, stopwords=stopwords, stemmer=stemmer_fn)

# If you want the tokenizer to return strings instead of token ids, you can do this
query_token_strs = bm25s.tokenize(queries, return_ids=True)

# You can use a different corpus for retrieval, e.g., titles instead of full docs
titles = ["About Cat", "About Dog", "About Bird", "About Fish"]

# You can also choose to only return the documents and omit the scores
results = retriever.retrieve(query_token_ids, k=2, return_as="documents")

# The documents are returned as a numpy array of shape (n_queries, k)
for i in range(results.shape[1]):
    print(f"Rank {i+1}: {results[0, i]}")

Split strings:   0%|          | 0/2 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/2 [00:00<?, ?it/s]

Split strings:   0%|          | 0/2 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/2 [00:00<?, ?it/s]

Rank 1: {'id': 0, 'text': 'a cat is a feline and likes to purr'}
Rank 2: {'id': 1, 'text': "a dog is the human's best friend and loves to play"}


In [58]:
results

array([[{'id': 0, 'text': 'a cat is a feline and likes to purr'},
        {'id': 1, 'text': "a dog is the human's best friend and loves to play"}],
       [{'id': 2, 'text': 'a bird is a beautiful animal that can fly'},
        {'id': 1, 'text': "a dog is the human's best friend and loves to play"}]],
      dtype=object)

In [51]:
corpus[results[1][0]]

IndexError: index 1 is out of bounds for axis 0 with size 1